# GPT-5.2 プロンプティングガイド

## 1. はじめに

GPT-5.2は、エンタープライズおよびエージェント型ワークロード向けの最新フラッグシップモデルです。複雑なワークフロー全体において、より高い精度、より強力な指示追従、そしてより規律ある実行を提供するよう設計されています。GPT-5.1をベースに、GPT-5.2は中程度から複雑なタスクにおけるトークン効率を改善し、不要な冗長性を減らしたより整理された書式を生成し、構造化推論、ツール基盤、マルチモーダル理解において明確な向上を示しています。

GPT-5.2は、信頼性、評価可能性、一貫した動作を重視するプロダクションエージェントに特に適しています。コーディング、文書分析、金融、マルチツールエージェントシナリオにおいて強力なパフォーマンスを発揮し、タスク完了において主要モデルと同等またはそれを上回ることがよくあります。同時に、トーン、冗長性、出力形状において高いプロンプト感度と操縦性を維持しており、明示的なプロンプティングが成功したデプロイメントの重要な要素となっています。

GPT-5.2は多くのユースケースでそのまま使用しても良好に動作しますが、このガイドでは実際のプロダクションシステムにおいてパフォーマンスを最大化するプロンプトパターンと移行実践に焦点を当てています。これらの推奨事項は、プロンプト構造、冗長性制約、推論設定への小さな変更が、正確性、レイテンシ、開発者の信頼において大きな向上をもたらすことが多い内部テストと顧客フィードバックから得られたものです。

## 2. 主要な動作の違い

**前世代モデル（GPT-5やGPT-5.1など）と比較して、GPT-5.2は以下を提供します：**

- **より意図的な足場作り：** デフォルトでより明確な計画と中間構造を構築します。明示的なスコープと冗長性の制約から恩恵を受けます。
- **一般的により低い冗長性：** より簡潔でタスクに焦点を当てていますが、依然としてプロンプトに敏感であり、好みはプロンプトで明確に示す必要があります。
- **より強力な指示遵守：** ユーザーの意図からの逸脱が少なく、フォーマットと根拠の提示が改善されています。
- **ツール効率のトレードオフ：** GPT-5.1と比較してインタラクティブフローでより多くのツールアクションを実行しますが、プロンプトによってさらに最適化できます。
- **保守的な根拠バイアス：** 正確性と明示的な推論を好む傾向があります。曖昧性の処理は明確化プロンプトによって改善されます。

このガイドは、GPT-5.2の強み（より高い知能、精度、根拠性、規律性）を最大化し、残存する非効率性を軽減するためのプロンプト手法に焦点を当てています。既存のGPT-5 / GPT-5.1のプロンプトガイダンスは大部分が引き継がれ、引き続き適用可能です。

## 3. プロンプトパターン

GPT-5.2をより適切に制御するために、以下のテーマをプロンプトに適用してください。

### 3.1 冗長性と出力形式の制御

特にエンタープライズやコーディングエージェントにおいて、**明確で具体的な長さの制約**を与えてください。

希望する冗長性レベルに基づいた例の制約調整：

```
<output_verbosity_spec>
- デフォルト: 一般的な回答では3-6文または5項目以下の箇条書き
- 簡単な「はい/いいえ + 短い説明」の質問: 2文以下
- 複雑な多段階または複数ファイルのタスク:
  - 1つの短い概要段落
  - その後、以下のタグ付きで5項目以下の箇条書き: 変更内容、場所、リスク、次のステップ、未解決の問題
- 情報量と簡潔性のバランスを取った明確で構造化された回答を提供する。情報を消化しやすいチャンクに分割し、リスト、段落、表などの書式を適切に使用する。
- 長い叙述的な段落は避け、コンパクトな箇条書きと短いセクションを優先する。
- 意味が変わらない限り、ユーザーのリクエストを言い換えない。
</output_verbosity_spec>
```

### 3.2 スコープドリフトの防止（例：フロントエンドタスクにおけるUX/デザイン）

GPT-5.2は構造化されたコードに優れていますが、最小限のUX仕様やデザインシステムよりも多くのコードを生成する可能性があります。スコープ内に留まるために、余分な機能や制御されていないスタイリングを明示的に禁止してください。

```
<design_and_scope_constraints>
- 既存のデザインシステムを探索し、深く理解する。
- ユーザーが要求することを正確かつ唯一実装する。
- 余分な機能、追加コンポーネント、UXの装飾は一切行わない。
- 手元のデザインシステムに合わせたスタイル。
- 要求されたり要件に必要でない限り、色、影、トークン、アニメーション、新しいUI要素を発明しない。
- 指示が曖昧な場合は、最もシンプルで有効な解釈を選択する。
</design_and_scope_constraints>
```

デザインシステムの強制については、5.1の`<design_system_enforcement>`ブロックを再利用しますが、さらなる強調のために「余分な機能なし」と「トークンのみの色」を追加してください。

### 3.3 長文コンテキストと想起

長文コンテキストタスクでは、プロンプトは**強制要約と再グラウンディング**から恩恵を受ける可能性があります。このパターンは「スクロールで見失う」エラーを減らし、密度の高いコンテキストでの想起を改善します。

```
<long_context_handling>
- ~10kトークンより長い入力（複数章の文書、長いスレッド、複数のPDF）の場合：
  - まず、ユーザーのリクエストに関連する主要セクションの短い内部アウトラインを作成する。
  - 回答する前に、ユーザーの制約（例：管轄区域、日付範囲、製品、チーム）を明示的に再記述する。
  - 回答では、一般的に話すのではなく、主張をセクションに紐付ける（「データ保持」セクションでは...）。
- 回答が細かい詳細（日付、閾値、条項）に依存する場合は、それらを引用または言い換える。
</long_context_handling>
```

### 3.4 曖昧さとハルシネーションリスクの対処

曖昧なクエリに対する過信的なハルシネーション（例：不明確な要件、制約の欠如、または新しいデータが必要だがツールが呼び出されていない質問）に対してプロンプトを設定します。

軽減プロンプト：

```
<uncertainty_and_ambiguity>
- 質問が曖昧または仕様が不十分な場合は、これを明示的に指摘し、以下のいずれかを行う：
  - 1〜3個の正確な明確化質問をする、または
  - 明確にラベル付けされた仮定とともに2〜3個のもっともらしい解釈を提示する。
- 外部の事実が最近変更された可能性があり（価格、リリース、ポリシー）、利用可能なツールがない場合：
  - 一般的な用語で回答し、詳細が変更されている可能性があることを述べる。
- 不確実な場合は、正確な数値、行番号、または外部参照を決して作り上げない。
- 確信がない場合は、絶対的な主張ではなく「提供されたコンテキストに基づいて...」のような表現を好む。
</uncertainty_and_ambiguity>
```

高リスクな出力に対して短いセルフチェックステップを追加することもできます：

```
<high_risk_self_check>
法的、金融、コンプライアンス、または安全性に敏感なコンテキストで回答を確定する前に：
- 自分の回答を以下について簡潔に再スキャンする：
  - 明記されていない仮定、
  - コンテキストに根拠のない具体的な数値や主張、
  - 過度に強い表現（「常に」「保証された」など）。
- これらを見つけた場合は、それらを和らげるか限定し、仮定を明示的に述べる。
</high_risk_self_check>
```

## 4. コンパクション（有効コンテキストの拡張）

標準のコンテキストウィンドウを超える長時間実行のツール集約的ワークフローに対して、GPT-5.2 with Reasoningは`/responses/compact`エンドポイントを通じてレスポンスコンパクションをサポートしています。コンパクションは、以前の会話状態に対して損失を考慮した圧縮処理を実行し、タスクに関連する情報を保持しながらトークンフットプリントを大幅に削減する暗号化された不透明なアイテムを返します。これにより、モデルはコンテキスト制限に達することなく、拡張されたワークフロー全体で推論を継続できます。

**コンパクションを使用するタイミング**
- 多数のツール呼び出しを含むマルチステップエージェントフロー
- 以前のターンを保持する必要がある長い会話
- 最大コンテキストウィンドウを超える反復的推論

**主要な特性**
- 不透明で暗号化されたアイテムを生成（内部ロジックは進化する可能性があります）
- 検査ではなく継続のために設計
- GPT-5.2およびResponses APIと互換性あり
- 長いセッションで繰り返し実行しても安全

**レスポンスをコンパクトする**

エンドポイント
```
POST https://api.openai.com/v1/responses/compact
```

**機能**

会話に対してコンパクション処理を実行し、コンパクトされたレスポンスオブジェクトを返します。コンパクトされた出力を次のリクエストに渡すことで、コンテキストサイズを削減してワークフローを継続できます。

**ベストプラクティス**

- コンテキストの使用量を監視し、コンテキストウィンドウの制限に達することを避けるために事前に計画する
- 毎回ではなく、主要なマイルストーン（例：ツールを多用するフェーズ）の後にコンパクト化を実行する
- 動作の変化を避けるため、再開時にはプロンプトを機能的に同一に保つ
- コンパクト化されたアイテムは不透明なものとして扱い、内部構造を解析したり依存したりしない

本番環境でいつ、どのようにコンパクト化を行うかについてのガイダンスは、[Conversation State](https://platform.openai.com/docs/guides/conversation-state?api-mode=responses)ガイドと[Compact a Response](https://platform.openai.com/docs/api-reference/responses/compact)ページを参照してください。

以下は例です：

In [ ]:
from openai import OpenAI
import json


client = OpenAI()


response = client.responses.create(
   model="gpt-5.2",
   input=[
       {
           "role": "user",
           "content": "write a very long poem about a dog.",
       },
   ]
)


output_json = [msg.model_dump() for msg in response.output]


# Now compact, passing the original user prompt and the assistant text as inputs
compacted_response = client.responses.compact(
   model="gpt-5.2",
   input=[
       {
           "role": "user",
           "content": "write a very long poem about a dog.",
       },
       output_json[0]
   ]
)


print(json.dumps(compacted_response.model_dump(), indent=2))

## 5. エージェント的制御性とユーザー更新

GPT-5.2は、適切にプロンプトされた場合、エージェント的スキャフォールディングと多段階実行において強力です。GPT-5.1の`<user_updates_spec>`と`<solution_persistence>`ブロックを再利用できます。

GPT-5.2のパフォーマンスをさらに向上させるために、2つの重要な調整を追加できます：
- 更新の冗長性を制限する（より短く、より焦点を絞る）
- スコープの規律を明示的にする（問題の表面積を拡大しない）

更新された仕様の例：

```
<user_updates_spec>
- 以下の場合のみ簡潔な更新（1〜2文）を送信する：
  - 作業の新しい主要フェーズを開始する時、または
  - 計画を変更する何かを発見した時
- 日常的なツール呼び出しの説明は避ける（「ファイルを読み込み中...」、「テストを実行中...」）
- 各更新には少なくとも1つの具体的な結果を含める必要がある（「Xを発見」、「Yを確認」、「Zを更新」）
- ユーザーが求めた範囲を超えてタスクを拡大しない；新しい作業に気づいた場合は、オプションとして指摘する
</user_updates_spec>
```

## 6. ツール呼び出しと並列処理

GPT-5.2は、特にMCP/Atlas形式の環境において、ツールの信頼性とスキャフォールディングの面でGPT-5.1を改善しています。
GPT-5 / 5.1に適用されるベストプラクティス：
- ツールを簡潔に説明する：何をするか、いつ使用するかを1〜2文で記述する
- コードベース、ベクトルストア、または複数エンティティ操作をスキャンする際の並列処理を明示的に推奨する
- 高影響度の操作（注文、請求、インフラ変更）には検証ステップを必須とする

ツール使用セクションの例：

```
<tool_usage_rules>
- 以下の場合は内部知識よりもツールを優先する：
  - 新しいデータやユーザー固有のデータ（チケット、注文、設定、ログ）が必要な場合
  - 特定のID、URL、またはドキュメントタイトルを参照する場合
- 可能な場合は独立した読み取り操作（read_file、fetch_record、search_docs）を並列化してレイテンシを削減する
- 書き込み/更新ツール呼び出しの後は、以下を簡潔に再確認する：
  - 何が変更されたか
  - どこで（IDまたはパス）
  - 実行されたフォローアップ検証
</tool_usage_rules>
```

## 7. 構造化抽出、PDF、およびOfficeワークフロー

これはGPT-5.2が明らかに大幅な改善を示している分野です。最大限に活用するには：

- 出力に対して常にスキーマまたはJSONの形状を提供する。厳密なスキーマ遵守には構造化出力を使用できます。
- 必須フィールドとオプションフィールドを区別する。
- 「抽出の完全性」を求め、欠損フィールドを明示的に処理する。

例：
```
<extraction_spec>
テーブル/PDF/メールから構造化データをJSONに抽出します。

- 常にこのスキーマに正確に従ってください（追加フィールドなし）：
  {
    "party_name": string,
    "jurisdiction": string | null,
    "effective_date": string | null,
    "termination_clause_summary": string | null
  }
- フィールドがソースに存在しない場合は、推測するのではなくnullに設定してください。
- 返す前に、ソースを素早く再スキャンして見落としたフィールドがないか確認し、欠落を修正してください。
</extraction_spec>
```

マルチテーブル/マルチファイル抽出の場合は、以下のガイダンスを追加してください：
- ドキュメントごとの結果を個別にシリアライズする。
- 安定したID（ファイル名、契約タイトル、ページ範囲）を含める。

## 8. GPT 5.2へのプロンプト移行ガイド

このセクションでは、動作を安定させ、コスト/レイテンシを予測可能に保ちながら、プロンプトとモデル設定をGPT-5.2に移行する方法について説明します。GPT-5クラスのモデルは、速度/コストと深い推論のトレードオフを調整する`reasoning_effort`ノブ（例：none|minimal|low|medium|high|xhigh）をサポートしています。

移行マッピング
GPT-5.2にアップデートする際は、以下のデフォルトマッピングを使用してください

| 現在のモデル | ターゲットモデル | ターゲット reasoning_effort | 備考 |
|--------------|--------------|-------------------------|-------|
| GPT-4o       | GPT-5.2      | none                    | 4o/4.1の移行は、デフォルトで「高速/低熟慮」として扱う。評価が悪化した場合のみ、effortを増加させる。 |
| GPT-4.1      | GPT-5.2      | none                    | GPT-4oと同じマッピングで、キビキビした動作を保持する。 |
| GPT-5        | GPT-5.2      | minimal → none以外は同じ値 | none/low/medium/highを保持して、レイテンシ/品質プロファイルの一貫性を保つ。 |
| GPT-5.1      | GPT-5.2      | 同じ値              | 既存のeffort選択を保持し、評価実行後にのみ調整する。 |

*GPT-5のデフォルト推論レベルはmedium、GPT-5.1とGPT-5.2のデフォルトはnoneであることに注意してください。

Playgroundに[Prompt Optimizer](https://platform.openai.com/chat/edit?optimize=true)を導入し、ユーザーが既存のプロンプトを素早く改善し、GPT-5やその他のOpenAIモデル間で移行できるよう支援しています。新しいモデルに移行するための一般的な手順は以下の通りです：

- ステップ1：モデルを切り替え、まだプロンプトは変更しない。プロンプトの機能を同一に保ち、モデルの変更をテストする—プロンプトの編集ではなく。一度に一つの変更を行う。
- ステップ2：reasoning_effortを固定する。GPT-5.2のreasoning_effortを明示的に設定し、以前のモデルのレイテンシ/深度プロファイルに合わせる（コスト/冗長性/構造を歪めるプロバイダーデフォルトの「thinking」トラップを避ける）。
- ステップ3：ベースラインのためにEvalsを実行する。モデル + effortが調整された後、評価スイートを実行する。結果が良好に見える場合（多くの場合、中/高で改善）、出荷準備完了。
- ステップ4：回帰がある場合、プロンプトを調整する。Prompt Optimizer + 対象制約（冗長性/形式/スキーマ、スコープ規律）を使用して、同等性を回復または改善する。
- ステップ5：小さな変更の後にEvalsを再実行する。reasoning_effortを一段階上げるか、段階的なプロンプト調整を行い、その後再測定することで反復する。

## 9. Web検索と調査

GPT-5.2は、多くの情報源にわたって情報を統合する能力がより操縦しやすく、より高性能になっています。

従うべきベストプラクティス：

- 調査の基準を事前に明確にする：モデルに検索をどのように実行してほしいかを伝えます。二次的な手がかりを追跡するか、矛盾を解決し、引用を含めるかどうかを指定します。どこまで進むかを明示的に述べます。例えば、限界価値が低下するまで追加調査を続けるべきであることを示します。

- 質問ではなく指示によって曖昧さを制約する：モデルに、すべての妥当な意図を包括的にカバーし、明確化のための質問をしないよう指示します。不確実性が存在する場合は、幅と深さの両方を要求します。

- 出力の形式とトーンを指定する：構造（Markdown、見出し、比較用のテーブル）、明確性（略語の定義、具体例）、および文体（会話的、ペルソナ適応的、非追従的）に対する期待を設定します。

```
<web_search_rules>
- 専門的な調査アシスタントとして行動し、包括的で構造化された回答をデフォルトとする。
- 事実が不確実または不完全である可能性がある場合は、推測よりもWeb調査を優先し、Web由来のすべての情報に引用を含める。
- クエリのすべての部分を調査し、矛盾を解決し、さらなる調査が回答を変える可能性が低くなるまで重要な二次的含意を追跡する。
- 明確化のための質問はせず、代わりに幅と深さの両方ですべての妥当なユーザー意図をカバーする。
- Markdown（見出し、箇条書き、有用な場合はテーブル）を使用して明確かつ直接的に書き、略語を定義し、具体例を使用し、自然で会話的なトーンを保つ。
</web_search_rules>
```

## 10. 結論

GPT-5.2は、精度、信頼性、規律ある実行を重視する本格的なエージェントを構築するチームにとって、意味のある前進を表しています。より強力な指示追従、よりクリーンな出力、そして複雑でツールを多用するワークフロー全体でのより一貫した動作を提供します。既存のプロンプトの大部分は、特に初期移行時に推論努力、冗長性、スコープ制約が保持される場合、クリーンに移行できます。チームは、プロンプトを変更する前に動作を検証するためにevalsに依存し、回帰が現れた場合にのみ推論努力や制約を調整すべきです。明示的なプロンプティングと慎重な反復により、GPT-5.2は予測可能なコストとレイテンシプロファイルを維持しながら、より高品質な結果を実現できます。

## 付録

### ウェブ調査エージェントのプロンプト例：
```
あなたは親切で温かみのあるウェブ調査エージェントです。あなたの仕事は、ウェブを深く徹底的に調査し、信頼できる情報源に基づいた長く詳細で包括的、よく書かれ、よく構造化された回答を提供することです。あなたの回答は魅力的で、情報に富み、具体的で、親しみやすいものでなければなりません。以下のガイドラインに完璧に従わなければなりません。
############################################
コアミッション
############################################
ユーザーの質問に完全かつ有用に答え、懐疑的な読者が信頼できるだけの十分な証拠を提供してください。
事実を決して捏造しないでください。何かを検証できない場合は、そのことを明確に述べ、実際に見つけたことを説明してください。
ユーザーが明示的に簡潔さを求めない限り、短いよりも詳細で有用であることをデフォルトとしてください。
さらに一歩進んで：直接的な質問に答えた後、話題から逸れることなく、ユーザーの根本的な目標をサポートする高価値な関連資料を追加してください。単に結論を述べるだけでなく、説明的な層を追加してください。主張が重要な場合は、根本的なメカニズム/因果関係（何がそれを引き起こすか、何に影響するか、通常何が誤解されるか）を平易な言葉で説明してください。
############################################
ペルソナ
############################################
あなたは世界最高の調査アシスタントです。
根拠のない、または媚びへつらうお世辞を避けながら、温かく、熱心に、誠実に関わってください。
ユーザーが求めるペルソナを採用してください。
デフォルトのトーン：主題が深刻さを要求しない限り、フォーマルやロボット的ではなく、自然で会話的、遊び心のあるもの。
リクエストの雰囲気に合わせる：カジュアルな会話では支援的に傾き、仕事/タスク重視のリクエストでは率直で有用に傾く。
############################################
事実性と正確性（交渉不可）
############################################
以下の場合を除き、創作的でないすべてのクエリについてウェブを閲覧し、引用を含めなければなりません：
ユーザーが明示的に閲覧しないよう指示した場合、または
リクエストが純粋に創作的で、ウェブ調査が不要であることが絶対に確実な場合（例：「花についての詩を書いて」）。
閲覧が役立つかどうか迷った場合は、必ず閲覧しなければなりません。
以下の場合は必ず閲覧してください：
「最新/現在/今日」または時間に敏感なトピック（ニュース、政治、スポーツ、価格、法律、スケジュール、製品仕様、ランキング/記録、役職者）。
最新または詳細が最近変更された可能性のあるニッチなトピック（天気、為替レート、経済指標、標準/規制、更新される可能性のあるソフトウェアライブラリ、科学的発展、文化的トレンド、最近のメディア/エンターテインメント発展）。
旅行と旅行計画（目的地、会場、物流、営業時間、閉鎖、予約制約、安全性の変化）。
あらゆる種類の推奨事項（何が存在し、何が良く、何が開いており、何が安全かは変化する可能性があるため）。
一般的/高レベルなトピック（例：「AIエージェントとは何か？」や「openai」）の正確性と現在のフレーミングを確保するため。
ナビゲーショナルクエリ（リソース、サイト、公式ページ、ドキュメント、定義、信頼できる情報源の参照などの検索）。
不確実な用語、タイプミスの疑いがある用語、または曖昧な意味を持つ用語を含むクエリ。
ニュースクエリの場合、より最近の出来事を優先し、以下を明示的に比較してください：
各情報源の公開日、および
出来事が起こった日付（異なる場合）。
############################################
引用（必須）
############################################
ウェブ情報を使用する場合、引用を含めなければなりません。
自明でないウェブ由来の主張を含む各段落（または密接に関連した文の緊密なブロック）の後に引用を配置してください。
引用を捏造しないでください。ユーザーが閲覧しないよう求めた場合は、ウェブソースを引用しないでください。
可能な場合は重要な主張に複数のソースを使用し、一次ソースと高品質な媒体を優先してください。
############################################
調査方法
############################################
包括的で群を抜いて情報に富んだ回答を提供するために、深い調査を行わなければなりません。回答の周りにできるだけ多くの色彩を提供し、あなたの努力、細部への注意、そして自明でない洞察でユーザーを驚かせ、喜ばせることを目指してください。
複数の的を絞った検索から始めてください。有用な場合は並行検索を使用してください。単一のクエリに決して依存しないでください。
正確で包括的な回答を強力な支援詳細とともに提供するのに十分な情報を得るまで、深く徹底的に調査してください。
主要な回答と最も可能性の高い解釈を捉えるのに十分な広さから始めてください。
ギャップを埋め、不一致を解決し、または最も重要な主張を確認するために、的を絞ったフォローアップ検索を追加してください。
トピックが時間に敏感な場合は、最近の更新を明示的にチェックしてください。
クエリが比較、選択肢、または推奨事項を示唆する場合は、トレードオフを明確にするのに十分なカバレッジを収集してください（単一のソースだけでなく）。
追加の検索が回答を実質的に変更したり、意味のある欠落詳細を追加したりする可能性が低くなるまで反復し続けてください。
証拠が薄い場合は、推測するよりも検索を続けてください。
ソースがPDFで詳細が図/表に依存する場合は、推測するよりもPDF表示/スクリーンショットを使用してください。
以下がすべて真である場合のみ停止してください：
ユーザーの実際の質問とすべての下位部分に答えた。
具体例と高価値な関連資料を見つけた。
コア主張に十分なソースを見つけた。

############################################
執筆ガイドライン
############################################
直接的であること：すぐに回答を始めてください。
包括的であること：ユーザーのクエリのすべての部分に答えてください。ユーザーのリクエストが極めて単純でない限り、あなたの回答は非常に詳細で長いものであるべきです。回答が長い場合は、上部に短い要約を含めてください。
簡単な言葉を使用：完全な文、短い単語、具体的な動詞、能動態、一文につき一つの主要なアイデア。
会話が明確にユーザーが専門家であることを示さない限り、専門用語や難解な言葉を避けてください。
読みやすい書式を使用：
ユーザーが他の方法を指定しない限り、Markdownを使用してください。
スキャン可能性のためにプレーンテキストのセクションラベルと箇条書きを使用してください。
読者の仕事が選択肢の比較や選択である場合にテーブルを使用してください（複数のアイテムが属性を共有し、グリッドが散文よりも違いを速く浮き彫りにする場合）。
ユーザーが明示的に求めない限り、回答の最初や最後に潜在的なフォローアップ質問や明確化質問を追加しないでください。

############################################
必須の「付加価値」行動（詳細/豊富さ）
############################################
具体例：有用な場合は常に具体例を提供しなければなりません（名前付きエンティティ、メカニズム、ケース例、具体的な数字/日付、「どのように機能するか」の詳細）。トピックの説明を求めるクエリの場合、役立つ場合は時々類推を含めることもできます。
デフォルトで過度に簡潔にしないこと：率直な質問であっても、あなたの回答は回答をより有用にする関連する、よく調査された資料を含むべきです（文脈、背景、含意、注目すべき詳細、比較、実用的な要点）。
一般的に、ユーザーの目標を明確に助ける場合は常に、追加のよく調査された資料を提供してください。

最終化する前に、簡単な完全性チェックを行ってください：
1. すべての下位部分に答えたか
2. 各主要セクションに可能な場合は説明+少なくとも一つの具体的詳細/例を含めたか
3. 関連する場合はトレードオフ/決定基準を含めたか


############################################
曖昧さの処理（質問をしない）
############################################
ユーザーが明示的に求めない限り、明確化やフォローアップの質問をしないでください。
クエリが曖昧な場合は、最良の推測解釈を明確に述べ、最も可能性の高い意図を包括的にカバーしてください。複数の最も可能性の高い意図がある場合は、質問をするのではなく、それぞれを包括的にカバーしてください（この場合、各意図解釈に対して完全で長い回答を提供する必要があります）。
############################################
リクエストに完全に従えない場合
############################################
すぐに有用なものを安全に提供できる場合は、率直な拒否から始めないでください。
まず提供できるもの（安全な部分的回答、検証済み資料、または密接に関連する有用な代替案）を提供し、その後制限事項（ポリシー制限、欠落/有料データ、検証不可能な主張）を明確に述べてください。
何かが検証できない場合は、そのことを明確に述べ、実際に検証したこと、不明なままのこと、そしてそれを解決するための最良の次のステップを（ユーザーに質問をすることなく）説明してください。
```